<a href="https://colab.research.google.com/github/SantiagoTadini/TP2-Datos/blob/main/ModeloCatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import log_loss
import pandas as pd
import datetime as dt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from urllib.request import urlretrieve
!pip install catboost
import catboost as cat

from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split

     |████████████████████████████████| 65.8MB 63kB/s 


# https://www.kaggle.com/jordimoragas/kernel-catboost-python


In [2]:
url_train = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_train.csv'
url_test = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test.csv'
url_test_kaggle = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test_kaggle.csv'

data_train = pd.read_csv(url_train)
data_test = pd.read_csv(url_test)
data_test_kaggle = pd.read_csv(url_test_kaggle)

In [3]:
# Separamos los set de datos (data_train y data_test) de sus columnas 'Target'
# 'x' son los datos, 'y' es el target:
#   --> x_train son los datos con los que entrenar
#   --> y_train son las soluciones con las que comparar lo entrenado con x_train
#   --> x_test son los datos con los que probar
#   --> y_test son las soluciones con las que comparar lo probado con x_test
x_train = data_train.iloc[:, :-1]
y_train = data_train.iloc[:, -1]
x_test = data_test.iloc[:, :-1]
y_test = data_test.iloc[:, -1]

#Esto lo hizo chechu y esta bien, no cambiar.

#CATBOOST

In [ ]:
#Preprocesmos datos para CatBoost (Pasamos datos a Pool que es el formato que requiere CatBoost)
lista_str=['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Bureaucratic_Code',
       'Account_Created_Date', 'Source', 'Billing_Country', 'Account_Name',
       'Opportunity_ID', 'Account_Owner', 'Opportunity_Owner', 'Account_Type',
       'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Quote_Expiry_Date',
       'Last_Modified_Date', 'Last_Modified_By', 'Product_Family',
       'Product_Name', 'ASP_Currency', 'ASP', 'ASP_(converted_USD)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'TRF', 'Total_Amount_Currency',
       'Total_Amount', 'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount',
       'USD_Total_Amount']
      
# Calculamos índice (posición) de las categóricas para pasar a CatBoost
Pos=list()

for col in lista_str:
    Pos.append((x_train.columns.get_loc(col)))
Pos = np.where(x.dtypes != float)[0]

# Pasamos a clase Pool
pool=cat.Pool(x_train, y_train, cat_features=Pos)

In [ ]:
# Parámetros básicos
RE=12345 # Seed que utilizaremos para partición de folds y la parte random del modelo
esr=50 # Early Stopping Rounds (si en "esr" árboles no mejora test -> se detiene).
# El objetivo es ajustar el par iterations / leraning_rate de manera que antes de acabar se detenga por "esr"
# Si llegamos al número final de rondas -> subir número de árboles o bajar learning_rate
n_folds=5 # Folds para CV

params = {'loss_function': 'RMSE',
          'learning_rate': 0.001, # En sintonía al número de árboles (iterations). Si no acaba por "esr", bajarlo
          'depth': 5, # Profundidad de los árboles (poner valores entre 5 y 10, más alto -> más overfitting)
          'l2_leaf_reg': 15, # Regularización L2 (poner entre 3 y 20, más alto -> menos overfitting).
          'rsm': 0.4, # % de features para hacer cada split (más bajo: acelera la ejecución y reduce overfitting)
          'random_seed': RE}


In [ ]:
print('\nCatBoost CV...\n')
cv_catboost = cat.cv(pool, params,
                iterations=50000, # Número de rondas máximo (árboles). Poner valor alto y que pare por "esr"
                nfold=n_folds,
                early_stopping_rounds=esr,
                verbose=50, # Nos muestra la métrica train/test cada tantos árboles
                partition_random_seed=RE)



CatBoost CV...

0:	learn: 0.7485021	test: 0.7484893	best: 0.7484893 (0)	total: 329ms	remaining: 4h 34m 28s
50:	learn: 0.7213874	test: 0.7213276	best: 0.7213276 (50)	total: 12.3s	remaining: 3h 21m 18s
100:	learn: 0.6957621	test: 0.6956223	best: 0.6956223 (100)	total: 24.6s	remaining: 3h 22m 19s
150:	learn: 0.6715172	test: 0.6713443	best: 0.6713443 (150)	total: 36.6s	remaining: 3h 21m 20s
200:	learn: 0.6484655	test: 0.6482120	best: 0.6482120 (200)	total: 48.9s	remaining: 3h 21m 55s
250:	learn: 0.6264195	test: 0.6260453	best: 0.6260453 (250)	total: 1m 1s	remaining: 3h 22m 50s
300:	learn: 0.6057438	test: 0.6052625	best: 0.6052625 (300)	total: 1m 13s	remaining: 3h 23m 30s
350:	learn: 0.5861486	test: 0.5855648	best: 0.5855648 (350)	total: 1m 26s	remaining: 3h 23m 39s
400:	learn: 0.5676168	test: 0.5669099	best: 0.5669099 (400)	total: 1m 38s	remaining: 3h 23m 37s
450:	learn: 0.5500895	test: 0.5492772	best: 0.5492772 (450)	total: 1m 51s	remaining: 3h 23m 35s
500:	learn: 0.5335883	test: 0.53266

In [ ]:
# Obtenemos número óptimo de rondas
best_nrounds=int((cv_catboost.shape[0]-esr)/(1-1/n_folds))
best_nrounds

62437

In [ ]:
model_catboost = cat.CatBoostRegressor(
          iterations=best_nrounds,
          learning_rate=0.01,
          depth=6,
          l2_leaf_reg=10,
          rsm=0.5,
          random_seed=RE,
          verbose=200)

print('\nCatBoost Fit...\n')
model_catboost.fit(X=pool)



CatBoost Fit...

0:	learn: 0.4931231	total: 49.5ms	remaining: 51m 31s
200:	learn: 0.2680854	total: 6.17s	remaining: 31m 49s
400:	learn: 0.2461376	total: 12.5s	remaining: 32m 14s
600:	learn: 0.2387972	total: 19s	remaining: 32m 35s
800:	learn: 0.2340184	total: 25.3s	remaining: 32m 30s
1000:	learn: 0.2303045	total: 31.6s	remaining: 32m 18s
1200:	learn: 0.2271130	total: 38s	remaining: 32m 15s
1400:	learn: 0.2247305	total: 44.2s	remaining: 32m 4s
1600:	learn: 0.2228354	total: 50.5s	remaining: 31m 57s
1800:	learn: 0.2209055	total: 56.8s	remaining: 31m 52s
2000:	learn: 0.2191892	total: 1m 3s	remaining: 31m 47s
2200:	learn: 0.2175358	total: 1m 9s	remaining: 31m 46s
2400:	learn: 0.2159272	total: 1m 16s	remaining: 31m 43s
2600:	learn: 0.2145463	total: 1m 22s	remaining: 31m 40s
2800:	learn: 0.2131701	total: 1m 29s	remaining: 31m 37s
3000:	learn: 0.2118660	total: 1m 35s	remaining: 31m 33s
3200:	learn: 0.2105043	total: 1m 42s	remaining: 31m 34s
3400:	learn: 0.2091975	total: 1m 48s	remaining: 31m 2

In [ ]:
pred_cb =  model_catboost.predict(x_test)

errorcb = log_loss(model_test['Target'], pred_cb)
errorcb

7.845997028052109

In [48]:

catb = CatBoostRegressor(learning_rate=0.01,
          depth=5,
          l2_leaf_reg=10,
          rsm=0.5,
          random_seed=12345,
          n_estimators = 120,
          verbose=200)

catb.fit(x_train, y_train)


0:	learn: 0.4948701	total: 5.87ms	remaining: 699ms
119:	learn: 0.3326652	total: 611ms	remaining: 0us


In [49]:
preds = catb.predict(x_test)

error = log_loss(y_test, preds)
error

0.5740348848157726

In [50]:
# Sacamos el y_test_de_practica_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

stage_test_de_practica = y_test.to_frame()

oportunidades_test_de_practica['Target'] = stage_test_de_practica
y_test_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
y_test_agrupado_por_opp = y_test_agrupado_por_opp.reset_index()

In [51]:
# Sacamos el preds_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

oportunidades_test_de_practica['Target'] = preds

preds_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_agrupado_por_opp = preds_agrupado_por_opp.reset_index()

In [52]:
# Calculamos el error con groupby hecho
error = log_loss(y_test_agrupado_por_opp['Target'], np.nan_to_num(preds_agrupado_por_opp['Target'].to_numpy()))
error

0.5797218169028855

In [ ]:
#PREDICCION SOBRE EL SET DE TEST
# Sacamos el preds_test_agrupado_por_opp
url_test = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test.csv'
data_test = pd.read_csv(url_test) #se usa para predecir

preds_testcb = model_catboost.predict(data_test)

oportunidades_testGV = data_test['Opportunity_ID'].to_frame()

oportunidades_testGV['Target'] = preds_testcb

preds_test_agrupado_por_oppGV = oportunidades_testGV.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_test_agrupado_por_oppGV = preds_test_agrupado_por_oppGV.reset_index()

preds_test_agrupado_por_oppGV

,Opportunity_ID,Target
0,10689,0.764411
1,10690,0.684776
2,10691,0.114514
3,10692,0.100321
4,10693,1.034008
...,...,...
1562,12364,1.107709
1563,12365,0.434293
1564,12366,0.267807
1565,12367,-0.100164


In [ ]:
from google.colab import files

preds_test_agrupado_por_oppGV.to_csv('pred_catboost.csv', index=False, header=True)
files.download('pred_catboost.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>